In [11]:
%matplotlib inline
#LOAD LIBRARIES
import csv
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display
from sklearn import linear_model as sk
from sklearn.linear_model import SGDRegressor
from sklearn.utils import shuffle
import numpy as np

In [12]:
%%time
#CONNECT TO THE DATABASE
import sqlite3
disk_engine=sqlite3.connect('final.db')

Wall time: 111 ms


In [13]:
#CONNECTION TEST
df = pd.read_sql_query('SELECT * FROM data LIMIT 3', disk_engine)
df

,index,Year,Month,DayofMonth,CRSDepTime,UniqueCarrier,Origin,Dest,Cancelled
0,1,1987,10,14,730,PS,SAN,SFO,0
1,2,1987,10,15,730,PS,SAN,SFO,0
2,3,1987,10,17,730,PS,SAN,SFO,0


In [14]:
%%time
model = sk.SGDClassifier()
classes= np.array([0,1])
for i in range(0,100,10):
    X_train = pd.read_sql_query('SELECT DayofMonth, Month, CRSDepTime '
                            'FROM data '
                           'LIMIT 10 OFFSET %i; ' %i, disk_engine)
    
    y_train = pd.read_sql_query('SELECT Cancelled '
                           'FROM data '
                           'LIMIT 10 OFFSET %i; ' %i, disk_engine)
    
    #X_train, y_train = shuffle(chunk_x, chunk_y)
    model.partial_fit(X_train, y_train.values.ravel(),classes=classes )


Wall time: 378 ms


In [15]:
print model.predict([[1987, 10, 1505]])

[0]


# NEW TEST

In [16]:
from sklearn.linear_model import SGDClassifier
from sklearn.utils.class_weight import compute_class_weight
import random

from sklearn.utils.testing import assert_almost_equal
from sklearn.utils.testing import assert_true
from sklearn.utils.testing import assert_equal

In [28]:
%%time

#create a list with the features we wan to use
feature_cols = ['Year','Month','CRSDepTime']

#define the classifier
classes = np.array([0,1])
weights = {0: 1, 1: 10}
SGDClass = SGDClassifier(class_weight=weights, 
                         #loss='log',
                         alpha=0.01,
                         #l1_ratio=1,
                         #learning_rate='optimal',
                         shuffle=True,
                         #eta0=0.01
                         )

#compute_class_weight('auto', classes, y)
for i in range(0,1000000,1000):
    train = pd.read_sql_query('SELECT * '
                              'FROM data '
                              'LIMIT 1000 OFFSET %i;  '%i, disk_engine)
    
    X = train.loc[:,feature_cols]
    y = train.Cancelled
    
    SGDClass.partial_fit(X,y,classes=classes)  

Wall time: 3min 17s


In [29]:
%%time 

#load a test dataframe
test = pd.read_sql_query('SELECT * '
                        'FROM data '
                        'LIMIT 1000000 OFFSET 3000000 ', disk_engine)
X_new = test.loc[:,feature_cols]

Wall time: 11.2 s


In [30]:
#predict
new_pred_class = SGDClass.predict(X_new)
print new_pred_class

yes_count=0
for pred in new_pred_class:
    if pred==1:
        yes_count+=1
        
#see results
#print len(new_pred_class)
print yes_count
print 100*(len(test[test['Cancelled']==1]))/(float(len(test))),'% is the REAL percentage of cancellations'
print (yes_count*100)/(float(len(new_pred_class))),'% is the PREDICTED percentage of cancellations'

[0 0 1 ..., 1 1 1]
534533
0.4481 % is the REAL percentage of cancellations
53.4533 % is the PREDICTED percentage of cancellations


In [63]:
#example flight
print SGDClass.predict([[1987, 10, 1505]])

[0]


In [3]:
# CROSS VALIDATION IMPLEMENTATION - PERCENTAGE COMPARATION

In [10]:
%%time

#create a list with the features we wan to use
feature_cols = ['Month','DayofMonth','CRSDepTime']

#define the classifier
classes = np.array([0,1])

#define variables for the process
w1=20
ran = range(0, (7000000-1000))
print len(ran)
cond = False

#function to compute the new class weight for each iteration:
def compute_weight (real_per, pred_per, w1, cond):
    diff = pred_per-real_per
    
    if diff == 0.0:
        cond=True
    elif 0<(diff)<3:
        cond=True
    elif 3<=(diff)<6:
        w1+=2
    elif 6<=(diff)<10:
        w1+=4
    elif 10<=(diff)<15:
        w1+=6
    elif (diff)>=15:
        w1+=10
    elif -3<=(diff)<0:
        cond=True
    elif -6<=(diff)<-3:
        w1-=2
    elif -10<=(diff)<-6:
        w1-=4
    elif -15<=(diff)<-10:
        w1-=6
    elif (diff)<-15:
        w1-=10

    return w1,diff, cond

#start the loop with false condition
while cond == False:

    print "New round, w1=",w1
    #set weight balance
    weights = {0: 1, 1: w1}
    #initialize the classifier
    SGDClass = SGDClassifier(class_weight=weights, 
                         #loss='log',
                         alpha=1.0,
                         #l1_ratio=1,
                         #learning_rate='optimal',
                         shuffle=True,
                         #eta0=0.01
                         )
    
    #Loop to train the classifier
    for i in range(0,1000000,10000):
        r = random.choice(ran)
        #print r
        #query for a chunck of the dataset to partial fit
        train = pd.read_sql_query('SELECT * '
                                  'FROM data '
                                  'LIMIT 10000 OFFSET %i; '%r, disk_engine)
        
        #obtain the X and y, with the desired training features
        X = train.loc[:,feature_cols]
        y = train.Cancelled
        
        #Train the classifier
        SGDClass.partial_fit(X,y,classes=classes)
    print "Load test dataframe"
    #load a test dataframe
    test = pd.read_sql_query('SELECT * '
                        'FROM data '
                        'LIMIT 10000 OFFSET %i; '%r, disk_engine)
    #Obtain the desired features for the test
    X_new = test.loc[:,feature_cols]
    print "Making prediction"
    #make prediction
    new_pred_class = SGDClass.predict(X_new)
    
    #count the cancellations
    yes_count=0
    for pred in new_pred_class:
        if pred==1:
            yes_count+=1
        
    #save percentage of cancellations and calculate difference
    real_per = 100*(len(test[test['Cancelled']==1]))/float(len(test))
    pred_per = (yes_count*100)/(float(len(new_pred_class)))
    print real_per,", ", pred_per
    
    #compute the new weight
    w1,diff,cond = compute_weight (real_per, pred_per, w1, cond)
        
    print "Comprobation, diff=",diff,"cond=",cond
    
print "Process finished"
print "Real percentage of cancellations:",real_per
print "Predicted percentage of cancellations:",pred_per
print "Difference:",diff

6999000
New round, w1= 20


AttributeError: 'DataFrame' object has no attribute 'Cancelled'

In [17]:
#function to compute the new class weight for each iteration:
def compute_weight (real_per, pred_per, w1):
    diff = pred_per-real_per
    
    if 0<=(res)<=3:
        cond=True
    elif 3<(res)<=6:
        w1+=2
    elif 6<(res)<=10:
        w1+=4
    elif 10<(res)<=15:
        w1+=6
    elif (res)>15:
        w1+=10
    elif -3<=(res)<0:
        cond=True
    elif -6<=(res)<-3:
        w1-=2
    elif -10<=(res)<-6:
        w1-=4
    elif -15<=(res)<-10:
        w1-=6
    elif (res)<-15:
        w1-=10

    return w1  

4586008


In [42]:
def comp_accuracy (new_pred_class, test):
count=0
for i in range(len(new_pred_class)):
    if new_pred_class[i]==test.Cancelled[i]:
        count+=1
accuracy=(float(count)/len(test.Cancelled))*100
return accuracy

99.79


# CROSS VALIDATION IMPLEMENTATION - ACCURACY

In [25]:
#create a list with the features we wan to use
feature_cols = ['Month','DayofMonth','CRSDepTime']

#define the classes
classes = np.array([0,1])

#define variables for the process
w1=20

#number of rows we'll use
num_rows=30000
print "Train sample:",num_rows,"lines"
ran = range(0,(11260000-num_rows))#112600 in the total number of lines in the database
cond = False

#function to compute the new class weight for each iteration:
def compute_weight_2 (acc, w1, cond):
    
    if acc == 100:
        cond=True
    elif 95<=(acc)<100:
        cond=True
    elif 90<=(acc)<95:
        w1-=1
    elif 87<=(acc)<90:
        w1-=2
    elif 80<=(acc)<87:
        w1-=3
    elif (acc)<80:
        w1-=4

    return w1,cond


#function to compute accuracy
def comp_accuracy (new_pred_class, test):
    count=0
    for i in range(len(new_pred_class)):
        if new_pred_class[i]==test.Cancelled[i]:
            count+=1
    accuracy=(float(count)/len(test.Cancelled))*100
    return accuracy

#MAIN LOOP
#start the loop with false condition
while cond == False:

    print "New round, w1=",w1
    #set weight balance
    weights = {0: 1, 1: w1}
    #initialize the classifier
    SGDClass = SGDClassifier(class_weight=weights, 
                         #loss='log',
                         alpha=1.0,
                         #l1_ratio=1,
                         #learning_rate='optimal',
                         shuffle=True,
                         #eta0=0.01
                         )
    
    #Loop to train the classifier
    for i in range(0,num_rows,1000):
        r = random.choice(ran)
        #print r
        #query for a chunck of the dataset to partial fit
        train = pd.read_sql_query('SELECT * '
                                  'FROM data '
                                  'LIMIT 1000 OFFSET %i; '%r, disk_engine)
        
        #obtain the X and y, with the desired training features
        X = train.loc[:,feature_cols]
        y = train.Cancelled
        
        #Train the classifier
        SGDClass.partial_fit(X,y,classes=classes)
    print "Load test dataframe. Train sample: 10000 lines"
    #load a test dataframe
    test = pd.read_sql_query('SELECT * '
                        'FROM data '
                        'LIMIT 10000 OFFSET %i; '%r, disk_engine)
    #Obtain the desired features for the test
    X_new = test.loc[:,feature_cols]
    print "Making prediction"
    #make prediction
    new_pred_class = SGDClass.predict(X_new)
    
    #calculate accuracy
    accuracy= comp_accuracy (new_pred_class, test)
    
    #compute the new weight
    w1, cond = compute_weight_2 (accuracy, w1, cond)
        
    print "Comprobation, accuracy=",accuracy,"cond=",cond
    
print "Process finished"
print "Accuracy:",accuracy,"%"

Train sample: 30000 lines
New round, w1= 20
Load test dataframe. Train sample: 10000 lines
Making prediction
Comprobation, accuracy= 99.38 cond= True
Process finished
Accuracy: 99.38 %


In [26]:
#create a list with the features we wan to use
feature_cols = ['Month','DayofMonth','CRSDepTime']

#define the classes
classes = np.array([0,1])

#define variables for the process
w1=20

#number of rows we'll use
num_rows=300000
print "Train sample:",num_rows,"lines"
ran = range(0,(11260000-num_rows))#112600 in the total number of lines in the database
cond = False

#function to compute the new class weight for each iteration:
def compute_weight_2 (acc, w1, cond):
    
    if acc == 100:
        cond=True
    elif 95<=(acc)<100:
        cond=True
    elif 90<=(acc)<95:
        w1-=1
    elif 87<=(acc)<90:
        w1-=2
    elif 80<=(acc)<87:
        w1-=3
    elif (acc)<80:
        w1-=4

    return w1,cond


#function to compute accuracy
def comp_accuracy (new_pred_class, test):
    count=0
    for i in range(len(new_pred_class)):
        if new_pred_class[i]==test.Cancelled[i]:
            count+=1
    accuracy=(float(count)/len(test.Cancelled))*100
    return accuracy

#MAIN LOOP
#start the loop with false condition
while cond == False:

    print "New round, w1=",w1
    #set weight balance
    weights = {0: 1, 1: w1}
    #initialize the classifier
    SGDClass = SGDClassifier(class_weight=weights, 
                         #loss='log',
                         alpha=1.0,
                         #l1_ratio=1,
                         #learning_rate='optimal',
                         shuffle=True,
                         #eta0=0.01
                         )
    
    #Loop to train the classifier
    for i in range(0,num_rows,1000):
        r = random.choice(ran)
        #print r
        #query for a chunck of the dataset to partial fit
        train = pd.read_sql_query('SELECT * '
                                  'FROM data '
                                  'LIMIT 1000 OFFSET %i; '%r, disk_engine)
        
        #obtain the X and y, with the desired training features
        X = train.loc[:,feature_cols]
        y = train.Cancelled
        
        #Train the classifier
        SGDClass.partial_fit(X,y,classes=classes)
    print "Load test dataframe. Test sample: 50000 lines"
    #load a test dataframe
    test = pd.read_sql_query('SELECT * '
                        'FROM data '
                        'LIMIT 50000 OFFSET %i; '%r, disk_engine)
    #Obtain the desired features for the test
    X_new = test.loc[:,feature_cols]
    print "Making prediction"
    #make prediction
    new_pred_class = SGDClass.predict(X_new)
    
    #calculate accuracy
    accuracy= comp_accuracy (new_pred_class, test)
    
    #compute the new weight
    w1, cond = compute_weight_2 (accuracy, w1, cond)
        
    print "Comprobation, accuracy=",accuracy,"cond=",cond
    
print "Process finished"
print "Accuracy:",accuracy,"%"

Train sample: 300000 lines
New round, w1= 20
Load test dataframe. Train sample: 50000 lines
Making prediction
Comprobation, accuracy= 2.93 cond= False
New round, w1= 16
Load test dataframe. Train sample: 50000 lines
Making prediction
Comprobation, accuracy= 99.394 cond= True
Process finished
Accuracy: 99.394 %


In [27]:
#create a list with the features we wan to use
feature_cols = ['Month','DayofMonth','CRSDepTime']

#define the classes
classes = np.array([0,1])

#define variables for the process
w1=20

#number of rows we'll use
num_rows=3000000
print "Train sample:",num_rows,"lines"
ran = range(0,(11260000-num_rows))#112600 in the total number of lines in the database
cond = False

#function to compute the new class weight for each iteration:
def compute_weight_2 (acc, w1, cond):
    
    if acc == 100:
        cond=True
    elif 95<=(acc)<100:
        cond=True
    elif 90<=(acc)<95:
        w1-=1
    elif 87<=(acc)<90:
        w1-=2
    elif 80<=(acc)<87:
        w1-=3
    elif (acc)<80:
        w1-=4

    return w1,cond


#function to compute accuracy
def comp_accuracy (new_pred_class, test):
    count=0
    for i in range(len(new_pred_class)):
        if new_pred_class[i]==test.Cancelled[i]:
            count+=1
    accuracy=(float(count)/len(test.Cancelled))*100
    return accuracy

#MAIN LOOP
#start the loop with false condition
while cond == False:

    print "New round, w1=",w1
    #set weight balance
    weights = {0: 1, 1: w1}
    #initialize the classifier
    SGDClass = SGDClassifier(class_weight=weights, 
                         #loss='log',
                         alpha=1.0,
                         #l1_ratio=1,
                         #learning_rate='optimal',
                         shuffle=True,
                         #eta0=0.01
                         )
    
    #Loop to train the classifier
    for i in range(0,num_rows,10000):
        r = random.choice(ran)
        #print r
        #query for a chunck of the dataset to partial fit
        train = pd.read_sql_query('SELECT * '
                                  'FROM data '
                                  'LIMIT 10000 OFFSET %i; '%r, disk_engine)
        
        #obtain the X and y, with the desired training features
        X = train.loc[:,feature_cols]
        y = train.Cancelled
        
        #Train the classifier
        SGDClass.partial_fit(X,y,classes=classes)
    print "Load test dataframe. Train sample: 100000 lines"
    #load a test dataframe
    test = pd.read_sql_query('SELECT * '
                        'FROM data '
                        'LIMIT 100000 OFFSET %i; '%r, disk_engine)
    #Obtain the desired features for the test
    X_new = test.loc[:,feature_cols]
    print "Making prediction"
    #make prediction
    new_pred_class = SGDClass.predict(X_new)
    
    #calculate accuracy
    accuracy= comp_accuracy (new_pred_class, test)
    
    #compute the new weight
    w1, cond = compute_weight_2 (accuracy, w1, cond)
        
    print "Comprobation, accuracy=",accuracy,"cond=",cond
    
print "Process finished"
print "Accuracy:",accuracy,"%"

Train sample: 3000000 lines
New round, w1= 20
Load test dataframe. Train sample: 100000 lines
Making prediction
Comprobation, accuracy= 99.272 cond= True
Process finished
Accuracy: 99.272 %


In [31]:
#create a list with the features we wan to use
feature_cols = ['Month','DayofMonth','CRSDepTime']

#define the classes
classes = np.array([0,1])

#define variables for the process
w1=10

#number of rows we'll use
num_rows=10000000
print "Train sample:",num_rows,"lines"
ran = range(0,(11260000-num_rows))#112600 in the total number of lines in the database
cond = False

#function to compute the new class weight for each iteration:
def compute_weight_2 (acc, w1, cond):
    
    if acc == 100:
        cond=True
    elif 95<=(acc)<100:
        cond=True
    elif 90<=(acc)<95:
        w1-=1
    elif 87<=(acc)<90:
        w1-=2
    elif 80<=(acc)<87:
        w1-=3
    elif (acc)<80:
        w1-=4

    return w1,cond


#function to compute accuracy
def comp_accuracy (new_pred_class, test):
    count=0
    for i in range(len(new_pred_class)):
        if new_pred_class[i]==test.Cancelled[i]:
            count+=1
    accuracy=(float(count)/len(test.Cancelled))*100
    return accuracy

#MAIN LOOP
#start the loop with false condition
while cond == False:

    print "New round, w1=",w1
    #set weight balance
    weights = {0: 1, 1: w1}
    #initialize the classifier
    SGDClass = SGDClassifier(class_weight=weights, 
                         #loss='log',
                         alpha=1.0,
                         #l1_ratio=1,
                         #learning_rate='optimal',
                         shuffle=True,
                         #eta0=0.01
                         )
    
    #Loop to train the classifier
    for i in range(0,num_rows,10000):
        r = random.choice(ran)
        #print r
        #query for a chunck of the dataset to partial fit
        train = pd.read_sql_query('SELECT * '
                                  'FROM data '
                                  'LIMIT 10000 OFFSET %i; '%r, disk_engine)
        
        #obtain the X and y, with the desired training features
        X = train.loc[:,feature_cols]
        y = train.Cancelled
        
        #Train the classifier
        SGDClass.partial_fit(X,y,classes=classes)
    print "Load test dataframe. Train sample: 1000000 lines"
    #load a test dataframe
    test = pd.read_sql_query('SELECT * '
                        'FROM data '
                        'LIMIT 1000000 OFFSET %i; '%r, disk_engine)
    #Obtain the desired features for the test
    X_new = test.loc[:,feature_cols]
    print "Making prediction"
    #make prediction
    new_pred_class = SGDClass.predict(X_new)
    
    #calculate accuracy
    accuracy= comp_accuracy (new_pred_class, test)
    
    #compute the new weight
    w1, cond = compute_weight_2 (accuracy, w1, cond)
        
    print "Comprobation, accuracy=",accuracy,"cond=",cond
    
print "Process finished"
print "Accuracy:",accuracy,"%"

Train sample: 10000000 lines
New round, w1= 10
Load test dataframe. Train sample: 1000000 lines
Making prediction
Comprobation, accuracy= 97.1657 cond= True
Process finished
Accuracy: 97.1657 %
